In [1]:
library(dplyr)
library(faraway)


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Warning message:
"package 'faraway' was built under R version 4.4.2"


403 African Americans were interviewed in a study to understand the prevalence of obesity, diabetes, and other cardiovascular risk factors in central Virginia. Glycosolated hemoglobin (glyhb) greater than 7.0 is usually taken as a positive diagnosis of diabetes. We are interested in identifying the most important predicotrs that contribute to a high glyhb level in african american patience while also creating a predictive model that accurately identifies patience with diabetes. 

In [2]:
diabetes

,id,chol,stab.glu,hdl,ratio,glyhb,location,age,gender,height,weight,frame,bp.1s,bp.1d,bp.2s,bp.2d,waist,hip,time.ppn
,<labelled>,<labelled>,<labelled>,<labelled>,<labelled>,<labelled>,<fct>,<int>,<fct>,<int>,<int>,<fct>,<labelled>,<labelled>,<labelled>,<labelled>,<int>,<int>,<labelled>
1,1000,203,82,56,3.6,4.31,Buckingham,46,female,62,121,medium,118,59,NA,NA,29,38,720
2,1001,165,97,24,6.9,4.44,Buckingham,29,female,64,218,large,112,68,NA,NA,46,48,360
3,1002,228,92,37,6.2,4.64,Buckingham,58,female,61,256,large,190,92,185,92,49,57,180
4,1003,78,93,12,6.5,4.63,Buckingham,67,male,67,119,large,110,50,NA,NA,33,38,480
5,1005,249,90,28,8.9,7.72,Buckingham,64,male,68,183,medium,138,80,NA,NA,44,41,300
6,1008,248,94,69,3.6,4.81,Buckingham,34,male,71,190,large,132,86,NA,NA,36,42,195
7,1011,195,92,41,4.8,4.84,Buckingham,30,male,69,191,medium,161,112,161,112,46,49,720
8,1015,227,75,44,5.2,3.94,Buckingham,37,male,59,170,medium,NA,NA,NA,NA,34,39,1020
9,1016,177,87,49,3.6,4.84,Buckingham,45,male,69,166,large,160,80,128,86,34,40,300
